# PCA

In this project, wel'll run PCA/TSNE (t-snee) on some population genotype data. 

**Content**
1. Download genotype data
2. PCA

[Reference](https://www.youtube.com/watch?v=-PCKK_nwFdA&t=2069s)

In [6]:
from basic_tools import *
from pysam import VariantFile
import numpy as np
from sklearn import decomposition
import pandas as pd

## 1. (optional) Parsing _.vcf_ file
We'll use 1000 Genome reference panel genotype data and those files can be download through below link.  
- 1KG genotype data URL : http://ftp.1000genomes.ebi.ac.uk/vol1/ftp/release/20110521/  
- Population label data : https://www.internationalgenome.org/data-portal/population    

You can skip this part if you already have accessible genotype file or using included _geno_mtx.csv_ file.

In [200]:
vcf_in = VariantFile(vcf_fn)

[E::idx_find_and_load] Could not retrieve index file for '../data/ALL.chr22.phase1_release_v3.20101123.snps_indels_svs.genotypes.vcf.gz'


In [199]:
# (optional) check original .vcf file
variants_cnt = 0
for rec in vcf_in.fetch():
    if rec.id is not None: # .id : variants ID
        variants_cnt += 1

print('# of variants : %d' %variants_cnt) # >> 493717
print('# of samples : %d' %len(list(vcf_in.header.samples))) # >> 1092


[E::idx_find_and_load] Could not retrieve index file for '../data/ALL.chr22.phase1_release_v3.20101123.snps_indels_svs.genotypes.vcf.gz'
[W::vcf_parse] Contig '22' is not defined in the header. (Quick workaround: index the file with tabix.)


# of variants : 493717
# of samples : 1092


In [206]:
# parsing vcf to matrix
genotypes = []
samples = []
variants_ids = []
with VariantFile(vcf_fn) as vcf_reader:
    cnt = 0
    for record in vcf_reader:
        cnt += 1
        if cnt % 100 == 0: # 100 : downsampling ratio
            samples = [sample for sample in record.samples]
            alleles = [record.samples[x].allele_indices for x in record.samples]
            genotypes.append(alleles)
            variants_ids.append(record.id)
        if cnt % 49371 == 0: #493717 -> 49371개 ~ 10%
            print('%d%%...' %(cnt//4937), end=" ")
        # if cnt > 10000: # max downsampling index
        #     break
print("DONE")

[E::idx_find_and_load] Could not retrieve index file for '../data/ALL.chr22.phase1_release_v3.20101123.snps_indels_svs.genotypes.vcf.gz'
[W::vcf_parse] Contig '22' is not defined in the header. (Quick workaround: index the file with tabix.)


10 %... 20 %... 30 %... 40 %... 50 %... 60 %... 70 %... 80 %... 90 %... 100 %... 

In [207]:
# data check
genotypes = np.array(genotypes)
print("shape of downsampled genotype matrix : {}".format(genotypes.shape)) # (samples, variants, diploid)

if np.isnan(genotypes).sum() != 0:
    print("There are some missing values in genotype matrix")
if not np.array_equal(np.unique(genotypes), np.array([0, 1])): # check genotype matrix is comprised of (0, 1)
    print("Some genotyping values are out of bound.")


shape of downsampled genotype matrix : (4943, 1092, 2)


In [208]:
gt_mtx = np.sum(genotypes, axis=2).T # row : samples, col : variants, val : diploid info
df_sample = pd.DataFrame(samples, columns=['Sample'])
df_geno = pd.DataFrame(gt_mtx, columns=variants_ids)
df = pd.concat([df_sample, df_geno], axis=1)
df.head()

,Sample,rs144366698,rs200391621,rs78888200,rs200049935,rs182808734,rs147783986,rs202082800,rs185745570,rs151231161,...,rs147260888,rs6010062,rs186386126,rs73174436,rs6010073,rs181024981,rs180882000,rs147614277,rs192355741,rs191731586
0,HG00096,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,HG00097,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,HG00099,0,0,1,0,0,1,0,0,0,...,0,1,0,0,2,0,0,1,0,0
3,HG00100,0,0,2,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,HG00101,0,0,1,0,0,1,0,0,0,...,0,2,0,0,1,0,0,0,0,0


In [209]:
# load panel data
panel = pd.read_csv(panel_fn, sep='\t', usecols=[0, 2], names=['Sample', 'Population code'], header=None)
panel.head()

,Sample,Population code
0,HG00096,EUR
1,HG00097,EUR
2,HG00099,EUR
3,HG00100,EUR
4,HG00101,EUR


In [210]:
# merge panel and genotype data
df = df.merge(panel, on='Sample')
df.head()

,Sample,rs144366698,rs200391621,rs78888200,rs200049935,rs182808734,rs147783986,rs202082800,rs185745570,rs151231161,...,rs6010062,rs186386126,rs73174436,rs6010073,rs181024981,rs180882000,rs147614277,rs192355741,rs191731586,Population code
0,HG00096,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,EUR
1,HG00097,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,EUR
2,HG00099,0,0,1,0,0,1,0,0,0,...,1,0,0,2,0,0,1,0,0,EUR
3,HG00100,0,0,2,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,EUR
4,HG00101,0,0,1,0,0,1,0,0,0,...,2,0,0,1,0,0,0,0,0,EUR


In [211]:
# save genotype matrix
df.to_csv('./data/geno_mtx.csv', index=False)


## 2. Load gentype data

In [212]:
gt_fh = './data/geno_mtx.csv'

# load genotype matrix
gt = pd.read_csv(gt_fh)
gt.head()

,Sample,rs144366698,rs200391621,rs78888200,rs200049935,rs182808734,rs147783986,rs202082800,rs185745570,rs151231161,...,rs6010062,rs186386126,rs73174436,rs6010073,rs181024981,rs180882000,rs147614277,rs192355741,rs191731586,Population code
0,HG00096,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,EUR
1,HG00097,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,EUR
2,HG00099,0,0,1,0,0,1,0,0,0,...,1,0,0,2,0,0,1,0,0,EUR
3,HG00100,0,0,2,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,EUR
4,HG00101,0,0,1,0,0,1,0,0,0,...,2,0,0,1,0,0,0,0,0,EUR


In [213]:
non_snp_coln = ['Sample', 'Population code']
gt_snps = gt.drop(non_snp_coln, axis=1)
gt_matrix = gt_snps.to_numpy()
gt_matrix

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 1, ..., 1, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0]])

In [214]:
# PCA
pca = decomposition.PCA(n_components=2)
pca.fit(gt_matrix)
to_plot = pca.transform(gt_matrix)

# check pca matrix
print(pca.explained_variance_ratio_)
print(pca.singular_values_) 
print(to_plot.shape)

[0.08253525 0.05412034]
[188.93732058 152.99536559]
(1092, 2)


In [215]:
import altair as alt

In [216]:
df_plot = gt[non_snp_coln].copy()
df_plot['PC1'] = to_plot[:,0]
df_plot['PC2'] = to_plot[:,1]
df_plot.head()

,Sample,Population code,PC1,PC2
0,HG00096,EUR,0.312827,5.354349
1,HG00097,EUR,-0.679403,6.183329
2,HG00099,EUR,-0.420710,5.266720
3,HG00100,EUR,-0.805419,5.965095
4,HG00101,EUR,-0.771385,4.332076


In [217]:
alt.Chart(df_plot).mark_point().encode(
    x='PC1',
    y='PC2',
    color=alt.Color('Population code', scale=alt.Scale(scheme='category20'))
)

alt.Chart(...)